# Fasttext 12 分类

### Utils

In [53]:
import fasttext
import jieba
import time
import pandas as pd
import numpy as np

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [54]:
def word_cut(content, stopword_set):
    cutWords = [k for k in jieba.cut(content, True) if k not in stopword_set]
    return " ".join(cutWords)

def ft_eval_model(test_label_list, predict_label_list, className_list):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(test_label_list, predict_label_list)
    # 计算总体的平均Precision, Recall, f1, support
    total_p = np.average(p, weights=s)
    total_r = np.average(r, weights=s)
    total_f1 = np.average(f1, weights=s)
    total_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': className_list,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [total_p],
        u'Recall': [total_r],
        u'F1': [total_f1],
        u'Support': [total_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

### Data

In [55]:
train_df = pd.read_csv('data/text_train.txt',sep='\t',names=['label','context'])
train_df['flag']='train'

test_df = pd.read_csv('data/text_test.txt',sep='\t',names=['label','context'])
test_df['flag']='test'

ft_all_data = train_df.append(test_df).reset_index(drop=True)


labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(ft_all_data['label'])

ft_all_data['encodedLabel'] = y
ft_all_data['encodedLabel'] = ft_all_data['encodedLabel'].apply(lambda x: '__label__'+str(x))

ft_all_data['context'] = ft_all_data['context'].apply(lambda x:word_cut(x, stopword_set))

ft_all_data.query('flag==\'train\'')[['context','encodedLabel']].to_csv('data/ft_hw_train_df.txt',sep='\t', index=False, header=None)

ft_all_data.query('flag==\'test\'')[['context','encodedLabel']].to_csv('data/ft_hw_test_df.txt',sep='\t', index=False, header=None)

In [56]:
map_dict={'总体':'总体'}
for _, row in ft_all_data[['label','encodedLabel']].drop_duplicates().iterrows():
    map_dict[row['encodedLabel']] = row['label']

In [72]:
labelEncoder.classes_

array(['体育', '健康', '女人', '娱乐', '房地产', '教育', '文化', '新闻', '旅游', '汽车', '科技',
       '财经'], dtype=object)

In [73]:
map_dict

{'总体': '总体',
 '__label__3': '娱乐',
 '__label__11': '财经',
 '__label__4': '房地产',
 '__label__8': '旅游',
 '__label__10': '科技',
 '__label__0': '体育',
 '__label__1': '健康',
 '__label__5': '教育',
 '__label__9': '汽车',
 '__label__7': '新闻',
 '__label__6': '文化',
 '__label__2': '女人'}

### Train

In [57]:
"""
  训练一个监督模型, 返回一个模型对象

  @param input:           训练数据文件路径
  @param lr:              学习率
  @param dim:             向量维度
  @param ws:              cbow模型时使用
  @param epoch:           次数
  @param minCount:        词频阈值, 小于该值在初始化时会过滤掉
  @param minCountLabel:   类别阈值，类别小于该值初始化时会过滤掉
  @param minn:            构造subword时最小char个数
  @param maxn:            构造subword时最大char个数
  @param neg:             负采样
  @param wordNgrams:      n-gram个数
  @param loss:            损失函数类型, softmax, ns: 负采样, hs: 分层softmax
  @param bucket:          词扩充大小, [A, B]: A语料中包含的词向量, B不在语料中的词向量
  @param thread:          线程个数, 每个线程处理输入数据的一段, 0号线程负责loss输出
  @param lrUpdateRate:    学习率更新
  @param t:               负采样阈值
  @param label:           类别前缀
  @param pretrainedVectors: 预训练的词向量文件路径, 如果word出现在文件夹中初始化不再随机
  @return model object
"""
# label的名字，和面前定义的前缀要对应上
classifier = fasttext.train_supervised(input='data/ft_hw_train_df.txt', dim=100, epoch=10,
                                         lr=0.1, wordNgrams=2, loss='softmax', label="__label__")

classifier.save_model('models/ft_hw_classifier.model')

result = classifier.test('data/ft_hw_test_df.txt')
print(result)

(12000, 0.78375, 0.78375)


### Eval

In [59]:
fasttext_test_df_ = np.array(ft_all_data.query('flag==\'test\'')['context']).tolist()

test_label_list = [int(x.strip("__label__")) for x in  ft_all_data.query('flag==\'test\'')['encodedLabel']]
test_label_list[:5]

predict_label_list = [int(y.strip("__label__")) for x in fasttext_test_df_ for y in classifier.predict(x)[0]]
predict_label_list[:5]

[3, 0, 3, 3, 3]

In [60]:
pd.DataFrame(confusion_matrix(test_label_list, predict_label_list), 
             columns=labelEncoder.classes_,
             index=labelEncoder.classes_ )

,体育,健康,女人,娱乐,房地产,教育,文化,新闻,旅游,汽车,科技,财经
体育,963,1,0,19,0,1,3,4,3,2,2,2
健康,0,837,28,1,0,14,12,48,2,1,36,21
女人,8,80,645,127,0,8,70,28,7,17,9,1
娱乐,6,0,28,791,0,9,150,2,6,0,7,1
房地产,1,0,1,6,888,7,0,22,9,5,4,57
教育,1,7,6,26,0,877,9,58,0,1,6,9
文化,3,13,36,213,2,6,605,57,31,7,26,1
新闻,11,32,11,28,21,71,50,586,24,7,39,120
旅游,4,10,12,15,7,2,70,26,817,3,17,17
汽车,16,6,3,2,1,7,3,6,7,901,13,35


In [64]:
ft_eval_model(test_label_list, predict_label_list, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,体育,0.948768,0.96300,0.955831,1000
1,健康,0.815789,0.83700,0.826259,1000
2,女人,0.832258,0.64500,0.726761,1000
3,娱乐,0.641525,0.79100,0.708464,1000
4,房地产,0.910769,0.88800,0.899241,1000
5,教育,0.858962,0.87700,0.867887,1000
6,文化,0.608652,0.60500,0.606820,1000
7,新闻,0.583085,0.58600,0.584539,1000
8,旅游,0.876609,0.81700,0.845756,1000
9,汽车,0.929825,0.90100,0.915185,1000


In [48]:
# W2V + LR

,Label,Precision,Recall,F1,Support
0,体育,0.954545,0.945,0.949749,1000
1,健康,0.864370,0.803,0.832556,1000
2,女人,0.709324,0.776,0.741165,1000
3,娱乐,0.735632,0.768,0.751468,1000
4,房地产,0.874753,0.887,0.880834,1000
5,教育,0.887310,0.874,0.880605,1000
6,文化,0.593069,0.599,0.596020,1000
7,新闻,0.611729,0.605,0.608346,1000
8,旅游,0.843299,0.818,0.830457,1000
9,汽车,0.931911,0.917,0.924395,1000


# Lstm tanh -> relu

* 理论角度：
    - tanh($f$)导数$f^{'}=1-(f^2)$，取值范围是$(0,1]$，且非零值集中在原点附近；relu在输入为正数的时候导数为$1$，其他情况导数为$0$。lstm的输出门部分的参数矩阵为$W_{out}$，只看输出门部分的话，一个输入数据的第一个时间步是$f(W_{out}*x),x$是输入到输出门的数据，不是原始输入，最后一个时间步是$f(W_{out}*f(W_{out}*....))$，如果$W_{out}$有大于1的元素，用relu会导致梯度爆炸。
* 实践角度：
    - 可以替换，如果发现梯度爆炸，可以尝试梯度裁剪(clip)。

# 过/欠拟合

* 欠拟合：模型在训练集上的表现差，验证测试也差（拟合能力差），连训练数据都学不明白。
    - 模型：
        - 原因：模型过于简单。
        - 处理方式：加layer，unit，其他结构，换模型（增加模型复杂度）；调参（降低正则项权重）。
    - 数据：
        - 原因：数据量太少，数据有问题，特征不够。
        - 处理方式：加数据，查/改数据，加特征。
        
* 过拟合：模型在训练集上表现好，但验证测试集上表现较差（泛化能力差）。
    - 模型：
        - 原因：模型过于复杂。
        - 处理方式：简化模型，调参（增加正则项，增加正则项权重），dropout，减少训练epoch（早停）。
    - 数据：
        - 原因：数据少/训练测试分布不一致，特征过多。
        - 处理方式：加数据，查/改数据，减特征。